# AMAZON FINAL AUTOMATED SCRAPING ALGORITHM

In [15]:
#Loading Required Liabraries
import re
import csv 
import requests
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions

#user defined function for accessing the product review page number URL and automating the page movement for fetchng the reviews
def get_url(link):
    template = link
    #add term query to url
    url = template.format(link)
    
    #add page query placeholder
    url += '&pageNumber={}'
    return url


#user defined function for extractig target data from the Amazon review webpage
def extract_record(item):
    """Extract and Return data from a single record"""
    
    ##Reviewer Name
    names = item.find('span', {'class':'a-profile-name'}).text
    
    try:
        #Review Title
        title = item.find('a', {'class':'review-title-content'}).text.strip()
        review_title =title.lstrip('\n')
        review_title =title.rstrip('\n')
    except AttributeError:
        title = ''
        

    try:
        #Review date
        review_date = item.find('span', {'data-hook':'review-date'}).text[20:].strip()
    except AttributeError:
        review_date = ''
        

    try:
        #Rating 
        rating = item.find('i', {'class':'review-rating'}).text
        rating_received = rating[0:3]
        total_rating = rating[11]
    except AttributeError:
        rating_received = ''
        total_rating = ''
        
    try:
        #Review Content 
        review = item.find('span', {'data-hook' : 'review-body'}).text.strip('\n ')
        #pattern = 'Your browser does not support HTML5 video'
        reviews = re.sub(r"Your browser does not support HTML5 video.\n\n\n  \xa0", "",review)
    except AttributeError:
        reviews = ''
              
    try:
        #user help
        user_help = item.find('span', {'data-hook':'helpful-vote-statement'}).text.strip()
        if user_help == 'One person found this helpful':
            user_help = '1 person found this helpful'
        help_vote = int(user_help.split()[0])
    except AttributeError:
        help_vote = '0'
        
    result = (names, review_title,review_date, rating_received,total_rating, reviews,help_vote)
    return result

#user defined main function for performing web scraping of reviews from the amazon webpage
def main(link):
    """Run Main Program Routine"""
    webdriver_location = "C:/Users/sarth/.wdm/drivers/edgedriver/win64/89.0.774.68/msedgedriver.exe"
    options = EdgeOptions()
    options.use_chromium = True
    driver = Edge(options=options, executable_path=webdriver_location)

    records = []
    url = get_url(link)
    
    #loop for getting the url of next review page along with accessing that page using beautiful soup
    for page in range(1,4):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, "html.parser")
        results = soup.find_all('div', {'class' : 'a-section celwidget'})
    
    #creating a loop for extracting the target data from each of the user review card
        for item in results:
            record = extract_record(item)
            if record:
                 records.append(record)
    
    driver.close()
    
            #save data to csv file
    with open('IACI18X95T3C_Amazon.csv','w', newline='',encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Reviewer Name','Review Title','Review Date', 'Rating Received','Total Rating', 'Review_Content', 'Helpful vote statement'])
        writer.writerows(records)


In [16]:
#calling the main function with passing the link of first review page for a product
main('https://www.amazon.in/IFB-Inverter-Split-Copper-IACI18X95T3C/product-reviews/B07PNNWTMJ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')